In [ ]:
shp_idv = '/mnt/disk1/dist_to_club/loc_individuos.shp'

shp_clubs = '/mnt/disk1/dist_to_club/loc_clubs.shp'

rel_tbl = {
    "TABLE"    : '/mnt/disk1/dist_to_club/calcdata.xlsx',
    "REL_INDV" : 'idv_desp',
    "REL_CLUB" : 'clubs_desp'
}

tmpfolder = '/mnt/disk1/dist_to_club/cf_tvtime/'
out = '/mnt/disk1/dist_to_club/idvtraveltm.shp'

impedance_type = 'TravelTime'

In [ ]:
import pandas as pd
import geopandas as gp
import os
from glass.rd.shp import shp_to_obj
from glass.rd import tbl_to_obj
from glass.mob.esri import closest_facility
from glass.gp.prox import pd_near
from glass.wt.shp import df_to_shp

In [ ]:
# Open Files
idv = shp_to_obj(shp_idv)
clb = shp_to_obj(shp_clubs)

rel_idv = tbl_to_obj(rel_tbl["TABLE"], sheet=rel_tbl["REL_INDV"])
rel_clb = tbl_to_obj(rel_tbl["TABLE"], sheet=rel_tbl["REL_CLUB"])

In [ ]:
impedances = []

In [ ]:
tmcol = 'walktime' if impedance_type == 'WalkTime' else 'traveltime'

In [ ]:
for i, row in idv.iterrows():
    # Get related desports
    rdesp = rel_idv[rel_idv['iid'] == row['iid']]
    
    dsp = rdesp["desporto"].tolist()
    
    # Get clubs with these desports
    rclb = rel_clb[rel_clb["desporto"].isin(dsp)]
    
    clbl = rclb["clubid"].tolist()
    
    clbg = clb[clb["clubid"].isin(clbl)]
    
    if not clbg.shape[0]:
        impedances.append(-2)
        continue
    
    idf = gp.GeoDataFrame([row], crs='EPSG:3763', geometry='geometry')
    
    near_club = pd_near(idf, clbg)
    
    d_to_clb = near_club.iloc[0].dist
    
    if d_to_clb / 1000 > 40:
        impedances.append(-1)
        continue
    
    # Run closest facility
    cfshp = closest_facility(
        idf, 'iid',
        clbg, os.path.join(tmpfolder, f'cfi_{row["iid"]}.shp'),
        impedance=impedance_type, crs=3763
    )
    
    # Open Result
    cfdf = shp_to_obj(cfshp)
    
    ttime = cfdf.iloc[0][tmcol]
    
    impedances.append(ttime)

In [ ]:
idv[tmcol] = impedances

In [ ]:
df_to_shp(idv, out)